In [1]:
import numpy as np
import cv2 as cv
import pandas as pd
import os
from random import shuffle
from keras.models import Sequential
from keras.layers import Dense, Conv2D , Flatten , Dropout
from sklearn.model_selection import train_test_split

print(os.listdir('cell_images'))

Using TensorFlow backend.


['.DS_Store', 'Parasitized', 'Uninfected']


In [2]:
cell_data = [] #This will store the cell image path and 1/0 depending whether it is Parasitized or not.

infected_cell_path = os.listdir('cell_images/Parasitized')
uninfected_cell_path = os.listdir('cell_images/Uninfected')


for i in infected_cell_path:
    cell_data.append(['cell_images/Parasitized/' + i , 1])

for i in uninfected_cell_path:
    cell_data.append(['cell_images/Uninfected/' + i , 0])

shuffle(cell_data)
images = [data[0] for data in cell_data]
state = [data[1] for data in cell_data]

print(len(images) , len(state))

del cell_data

27560 27560


In [5]:
def get_image(path):
    '''Returns image matrix data in RGB format'''
    im = cv.imread(path , 1)
    im = cv.resize(im , (60 , 60))
    im = im/255 #Normalize the values in the image
    return im


X = []
y = []

for i in range(len(images)):
    
    try:
        
        X.append(get_image(images[i]))
        y.append(state[i])
    
    except:
        pass

print(len(X) , len(y))

27558 27558


In [6]:
X_train , X_test , y_train , y_test = train_test_split(X , y , test_size = 0.1 , random_state = 66)
X_train , X_val , y_train , y_val = train_test_split(X_train , y_train , test_size = 0.1 , random_state = 64)

In [7]:
X_train = np.array(X_train)
y_train = np.array(y_train)
X_val = np.array(X_val)
y_val = np.array(y_val)

In [14]:
#Setup CNN model

model = Sequential()

model.add(Conv2D(64 , kernel_size = 3 , activation='relu' , input_shape = (60 , 60 , 3)))
model.add(Conv2D(32 , kernel_size = 3 , activation='relu'))
model.add(Conv2D(16 , kernel_size = 3 , activation='relu'))
model.add(Flatten())
model.add(Dropout(0.1))
model.add(Dense(1 , activation = 'sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [17]:
model.fit(X_train , y_train , validation_data=(X_val , y_val) , epochs=10)

Train on 22321 samples, validate on 2481 samples
Epoch 1/10
22321/22321 [==============================] - 347s 16ms/step - loss: 0.6430 - acc: 0.6359 - val_loss: 0.7043 - val_acc: 0.8541
Epoch 2/10
22321/22321 [==============================] - 339s 15ms/step - loss: 0.4405 - acc: 0.8536 - val_loss: 0.6088 - val_acc: 0.6868
Epoch 3/10
22321/22321 [==============================] - 335s 15ms/step - loss: 0.3806 - acc: 0.8570 - val_loss: 0.2945 - val_acc: 0.9037
Epoch 4/10
22321/22321 [==============================] - 340s 15ms/step - loss: 0.2393 - acc: 0.9215 - val_loss: 0.2441 - val_acc: 0.9029
Epoch 5/10
22321/22321 [==============================] - 340s 15ms/step - loss: 0.1550 - acc: 0.9503 - val_loss: 0.2012 - val_acc: 0.9359
Epoch 6/10
22321/22321 [==============================] - 336s 15ms/step - loss: 0.1221 - acc: 0.9615 - val_loss: 0.2081 - val_acc: 0.9359
Epoch 7/10
22321/22321 [==============================] - 338s 15ms/step - loss: 0.1187 - acc: 0.9618 - val_loss: 0.2

In [18]:
del X_train, y_train , X_val , y_val

In [19]:
X_test = np.array(X_test)
y_test = np.array(y_test)

In [21]:
pred = np.rint(model.predict(X_test))

In [23]:
success = 0
for i in range(len(pred)):
    if(pred[i] == y_test[i]):
        success += 1

print(str(np.round((success*100)/len(pred),2))+'% of test cases were predicted successfully')

93.29% of test cases were predicted successfully


In [36]:
test_path = 'cell_images/Uninfected/C49P10thinF_IMG_20150724_102843_cell_64.png'
im = []
im.append(get_image(test_path))
im = np.array(im)
np.rint(model.predict(im)) #Returns 0 -----> Uninfected Cell (Correct answer)

array([[ 0.]], dtype=float32)